In [1]:
import datetime
import pandas as pd
import requests
import io
import json

url = 'http://gis.sutran.gob.pe/alerta_sutran/script_cgm/carga_xlsx.php'
headers = {'Accept': '/',
'Accept-Encoding': 'gzip, deflate',
'Accept-Language': 'en,es;q=0.9,en-GB;q=0.8',
'Connection': 'keep-alive',
'Content-Length': '9',
'Content-type': 'application/x-www-form-urlencoded; charset=UTF-8-SIG',
'Host': 'gis.sutran.gob.pe',
'Origin': 'http://gis.sutran.gob.pe',
'Referer': 'http://gis.sutran.gob.pe/alerta_sutran/',
'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Mobile Safari/537.36'}
body = {'tipo':'MAPA'}
req = requests.post(url, headers=headers, data=body)
now = datetime.datetime.now()

my_json = json.load(io.BytesIO(req.content))

restringido = {}
restringido['data'] = my_json['restringido']

interrumpido = {}
interrumpido['data'] = my_json['interrumpido']

with open('restringido.json', 'w') as f:
    json.dump(restringido, f)

with open('interrumpido.json', 'w') as f:
    json.dump(interrumpido, f)

df_r = pd.read_json('restringido.json', encoding = 'utf-8-sig', orient='split')
df_restringido = pd.json_normalize(df_r['properties'])

df_i = pd.read_json('interrumpido.json', encoding = 'utf-8-sig', orient='split')
df_interrumpido = pd.json_normalize(df_i['properties'])

frames = [df_restringido, df_interrumpido]
df_total = pd.concat(frames, ignore_index=True)
df_total = df_total[df_total['motivo'] == 'HUMANO']
df_total = df_total.replace('TRANSITO', 'TRÁNSITO', regex=True)
df_total = df_total.replace('"', '', regex=True)
df_total['style'] = df_total['estado'].str.split(' ').str[1].str.lower()
df_total.to_csv(f'{now}.csv', index=False)
df_total

,item,visualizacion_,tipo_alerta_,estado,fecha_evento,fecha_actualizacion,ubigeo,afectacion,evento,latitud,...,_sort_,_fecha_actualizacion_,cant_vehiculos_detenidos_pasajeros,cant_vehiculos_detenidos_mercancias,fuente,motivo,codigo_via,nombre_carretera,alerta_activa_24_horas,style
2,222,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,20/01/2023,21/01/2023 21:00 HORAS,MADRE DE DIOS/TAMBOPATA/LABERINTO,KM 382+200,(SECTOR LABERINTO) PARO INDEFINIDO SOLICITANDO...,-11.4068843,...,-11.4068843+-69.4784365,2023 21:00 HORAS+01+21+21:00,2,4,IIRSA SUR,HUMANO,PE-30C,CARRETERA URCOS - MAZUCO - IÑAPARI,SI,interrumpido
3,227,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,06/01/2023,21/01/2023 21:00 HORAS,MADRE DE DIOS/TAMBOPATA/LAS PIEDRAS,KM 511+800,(PUENTE MAVILA) PARO INDEFINIDO SOLICITANDO NU...,-11.931332,...,-11.931332+-69.11748,2023 21:00 HORAS+01+21+21:00,0,0,IIRSA SUR,HUMANO,PE-30C,CARRETERA URCOS - MAZUCO - IÑAPARI,SI,interrumpido
4,229,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,21/01/2023,21/01/2023 21:00 HORAS,MADRE DE DIOS/TAMBOPATA/LAS PIEDRAS,471,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-12.2828444,...,-12.2828444+-69.1554046,2023 21:00 HORAS+01+21+21:00,0,0,IIRSA SUR,HUMANO,PE-30C,CARRETERA URCOS - MAZUCO - IÑAPARI,SI,interrumpido
5,230,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,08/01/2023,21/01/2023 21:00 HORAS,MADRE DE DIOS/TAMBOPATA/TAMBOPATA,KM 402,(SAN BERNARDO) PARO INDEFINIDO SOLICITANDO NUE...,-12.6898325,...,-12.6898325+-69.4042951,2023 21:00 HORAS+01+21+21:00,0,0,IIRSA SUR,HUMANO,PE-30C,CARRETERA URCOS - MAZUCO - IÑAPARI,SI,interrumpido
6,145,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,07/01/2023,21/01/2023 21:00 HORAS,CUSCO/LA CONVENCION/SANTA ANA,KM 230,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-12.880884243624,...,-12.880884243624+-72.690784623809,2023 21:00 HORAS+01+21+21:00,0,1,DESPRCAR QUILLABAMBA,HUMANO,PE-28B,CARRETERA QUINUA - QUILLABAMBA - CUSCO,SI,interrumpido
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,242,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,19/01/2023,21/01/2023 21:00 HORAS,PIURA/AYABACA/PACAIPAMPA,KM 1862,SECTOR BELLAVISTA PARO INDEFINIDO SOLICITANDO ...,-4.9650302728176,...,-4.9650302728176+-79.538985681552,2023 21:00 HORAS+01+21+21:00,0,0,COE-MTC,HUMANO,PE-3N,LONGITUDINAL DE LA SIERRA NORTE,SI,interrumpido
104,243,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,19/01/2023,21/01/2023 21:00 HORAS,PIURA/HUANCABAMBA/HUARMACA,KM 60,PUENTE HUALAPAMPA PARO INDEFINIDO SOLICITANDO ...,-5.79078,...,-5.79078+-79.44051,2023 21:00 HORAS+01+21+21:00,0,0,COE-MTC,HUMANO,PE-04B,CARRETERA OLMOS - HUARMACA,SI,interrumpido
105,330,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,11/12/2022,21/01/2023 21:00 HORAS,UCAYALI/PADRE ABAD/PADRE ABAD,KM 390,SECTOR HUIPOCA. PARO INDEFINIDO SOLICITANDO NU...,-8.976077,...,-8.976077+-75.44483,2023 21:00 HORAS+01+21+21:00,0,0,CEOPOL,HUMANO,PE-5N,LONGITUDINAL DE LA SELVA NORTE,SI,interrumpido
106,331,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,19/01/2023,21/01/2023 21:00 HORAS,UCAYALI/PADRE ABAD/PADRE ABAD,KM 391,SECTOR HUIPOCA. PARO INDEFINIDO SOLICITANDO NU...,-8.9903469087213,...,-8.9903469087213+-75.452262878851,2023 21:00 HORAS+01+21+21:00,0,0,UD UCAYALI,HUMANO,PE-5N,LONGITUDINAL DE LA SELVA NORTE,SI,interrumpido
